In [1]:
# import numpy as np
# import pandas as  pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
# from catboost import CatBoostClassifier      
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import KFold
# import statistics
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score



import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

## feature engineered dfs 

In [2]:
new_train_df = pd.read_csv('new_train.csv')
new_test_df = pd.read_csv('new_test.csv')

## EDA

In [ ]:
# import seaborn as sns
# sns.countplot(new_train_df['outcome'],label="Count")

In [ ]:
# new_train_df.corr()

In [ ]:
# import matplotlib.pyplot as plt
# corr = new_train_df.iloc[:,:].corr()
# colormap = sns.diverging_palette(220, 10, as_cmap = True)
# plt.figure(figsize=(14,14))
# sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 12},
#             cmap = colormap, linewidths=0.1, linecolor='white')
# plt.title('Correlation of new_train Features', y=1.05, size=15)

## Stratified K-Fold with SMOTE

In [3]:

def training_model(model, num_splits, X, y):
    skfolds = StratifiedKFold(n_splits=num_splits)

    for train_index, test_index in skfolds.split(X, y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]        
    
        sm = SMOTE()
        X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

        
        model = model
        model.fit(X_train_oversampled, y_train_oversampled )  
        y_pred = model.predict_proba(X_test)
        
        print(f'Accuracy: {model.score(X_test, y_test)}')
#         print(f'f-score: {f1_score(y_test, y_pred)}')
        print("AUC : ", roc_auc_score(y_test,y_pred[:,1]))

## modelling without Backward elimination



In [4]:

X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
from sklearn.naive_bayes import BernoulliNB
from catboost import CatBoostClassifier
model = CatBoostClassifier(learning_rate=0.1, max_depth=3,random_state = 10)
num_splits = 10
training_model(model, num_splits, X_scaled, y)


NameError: name 'training_model' is not defined

## modelling with backward elimination

In [ ]:
X = new_train_df[['device', 'time', 'ip', 'num_bids', 'num_first_bids', 'num_last_bids', 'inst_resp', 'perc_inst_resp', 'auto parts', 'books and music', 'clothing', 'computers', 'furniture', 'home goods', 'jewelry', 'mobile', 'office equipment', 'sporting goods', 'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip']]
y=new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

def training_model(model, num_splits, X, y):
    skfolds = StratifiedKFold(n_splits=num_splits)

    for train_index, test_index in skfolds.split(X, y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]        
    
        sm = SMOTE()
        X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

        
        model = model
        model.fit(X_train_oversampled, y_train_oversampled )  
        y_pred = model.predict(X_test)
        
        print(f'Accuracy: {model.score(X_test, y_test)}')
        print(f'f-score: {f1_score(y_test, y_pred)}')
        print("AUC : ", roc_auc_score(y_test,y_pred[:,1]))        
        
model = CatBoostClassifier(learning_rate=0.1, max_depth=3,random_state = 10)
num_splits = 10
training_model(model, num_splits, X_scaled, y)

# hyperparameters tuning

## Catboost

In [9]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier

X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
sm = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.7, random_state=42)
X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)


#Instantiate CatBoostClassifier
cbc = CatBoostClassifier()
param_dist = { "learning_rate": np.linspace(0,0.2,5),"max_depth": randint(3, 10)}
cbc_random = RandomizedSearchCV(estimator = cbc, param_distributions = param_dist, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
cbc_random.fit(X_train_oversampled,y_train_oversampled)


NameError: name 'X_scaled' is not defined

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict_proba(test_features)
    pred = model.predict(test_features)
    
    accuracy = accuracy_score(test_labels, pred)
    auc_roc_score = roc_auc_score(test_labels,predictions[:,1])
    
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('AUC ROC = {:0.2f}%.'.format(auc_roc_score))
    
    return accuracy, auc_roc_score

In [ ]:
base_model = CatBoostClassifier()
base_model.fit(X_train_oversampled, y_train_oversampled)
base_accuracy = evaluate(base_model, X_test, y_test)
best_random = cbc_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

## ANN

### GridSearch with 4 hyperparam - weiquan run!



In [41]:

import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
sm = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
# split into input (X) and output (Y) variables
X = X_train_oversampled
Y = y_train_oversampled
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.914854 using {'batch_size': 10, 'epochs': 100, 'learn_rate': 0.001, 'momentum': 0.6}
0.741905 (0.075117) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.0}
0.745284 (0.072862) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.2}
0.740779 (0.085647) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.4}
0.759172 (0.062148) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.6}
0.728775 (0.088594) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.8}
0.724634 (0.117910) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.001, 'momentum': 0.9}
0.744908 (0.074381) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.01, 'momentum': 0.0}
0.756150 (0.081989) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.01, 'momentum': 0.2}
0.719022 (0.092673) with: {'batch_size': 10, 'epochs': 10, 'learn_rate': 0.01, 'momentum': 0.4}
0.729519 (0.091276) with: {'batch_siz

# Grid Search for Learn rate and momentum

In [39]:

import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
sm = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
# split into input (X) and output (Y) variables
X = X_train_oversampled
Y = y_train_oversampled
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.782452 using {'learn_rate': 0.01, 'momentum': 0.6}
0.634288 (0.022368) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.451458 (0.295540) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.706661 (0.054742) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.589930 (0.182340) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.657129 (0.083882) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.515724 (0.166210) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.501472 (0.128784) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.535504 (0.244883) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.565588 (0.150534) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.782452 (0.012026) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.558078 (0.123363) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.514238 (0.116878) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.652968 (0.144539) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.599025 (0.115265) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.595707 (0.193572) with: {'learn_rate': 

### RandomSearch with 4 hyperparameters -weiquan run!

In [42]:
import numpy
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
sm = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.7, random_state=42)
X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
# split into input (X) and output (Y) variables
X = X_train_oversampled
Y = y_train_oversampled
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
# random = RandomizedSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
random = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
random_result = random.fit(X, Y)
# summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
params = random_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.1min finished


Best: 0.915229 using {'momentum': 0.0, 'learn_rate': 0.1, 'epochs': 100, 'batch_size': 10}
0.692558 (0.115002) with: {'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 50, 'batch_size': 40}
0.885203 (0.049115) with: {'momentum': 0.2, 'learn_rate': 0.1, 'epochs': 100, 'batch_size': 10}
0.754413 (0.072634) with: {'momentum': 0.9, 'learn_rate': 0.01, 'epochs': 100, 'batch_size': 100}
0.896683 (0.032159) with: {'momentum': 0.4, 'learn_rate': 0.3, 'epochs': 100, 'batch_size': 10}
0.583791 (0.215883) with: {'momentum': 0.2, 'learn_rate': 0.2, 'epochs': 10, 'batch_size': 100}
0.882564 (0.052966) with: {'momentum': 0.9, 'learn_rate': 0.1, 'epochs': 100, 'batch_size': 10}
0.453939 (0.328517) with: {'momentum': 0.6, 'learn_rate': 0.3, 'epochs': 10, 'batch_size': 60}
0.684603 (0.145545) with: {'momentum': 0.8, 'learn_rate': 0.3, 'epochs': 50, 'batch_size': 80}
0.765030 (0.091477) with: {'momentum': 0.0, 'learn_rate': 0.1, 'epochs': 100, 'batch_size': 60}
0.638684 (0.082571) with: {'momentum': 0.9, 'l

## Models Created using the tuned hyperparameters 

In [52]:
#Create model using Randomsearch hyperparameters
def create_model(learn_rate=0.01, momentum=0,epochs=100,batch_size=10):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#create model using GridSearch hyperparamters

def create_model(learn_rate=0.001, momentum=0.6,epochs=100,batch_size=10):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#Include ACCURACY HERE 


## Keras Tuner Random Search



In [10]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten())

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

X = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train_df['outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
sm = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

tuner = kt.RandomSearch(model_builder,
                     objective='val_accuracy',
                    max_trials = 4
                       )
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train_oversampled, y_train_oversampled, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 64 and the optimal learning rate for the optimizer
is 0.01.



In [36]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_oversampled, y_train_oversampled, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))



Epoch 1/50
67/67 [==============================] - 0s 2ms/step - loss: 0.5829 - accuracy: 0.7795 - val_loss: 0.5310 - val_accuracy: 0.7228
Epoch 2/50
67/67 [==============================] - 0s 890us/step - loss: 0.3141 - accuracy: 0.8715 - val_loss: 0.4729 - val_accuracy: 0.7809
Epoch 3/50
67/67 [==============================] - 0s 838us/step - loss: 0.2507 - accuracy: 0.9043 - val_loss: 0.2667 - val_accuracy: 0.9157
Epoch 4/50
67/67 [==============================] - 0s 847us/step - loss: 0.2027 - accuracy: 0.9278 - val_loss: 0.2514 - val_accuracy: 0.9007
Epoch 5/50
67/67 [==============================] - 0s 827us/step - loss: 0.1714 - accuracy: 0.9371 - val_loss: 0.1284 - val_accuracy: 0.9588
Epoch 6/50
67/67 [==============================] - 0s 854us/step - loss: 0.1376 - accuracy: 0.9508 - val_loss: 0.1567 - val_accuracy: 0.9682
Epoch 7/50
67/67 [==============================] - 0s 847us/step - loss: 0.1485 - accuracy: 0.9550 - val_loss: 0.1137 - val_accuracy: 0.9569
Epoch 8/

In [60]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# from tf.keras.optimizers import SGD

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
X = X_train_oversampled
Y = y_train_oversampled
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.921437 using {'learn_rate': 0.3, 'momentum': 0.4}
0.888763 (0.059324) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.899335 (0.043818) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.895822 (0.049798) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.905517 (0.034849) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.871095 (0.057871) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.906430 (0.055439) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.806581 (0.055063) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.869283 (0.053607) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.884363 (0.070057) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.895810 (0.042154) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.856930 (0.047736) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.895820 (0.053047) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.810958 (0.100750) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.907288 (0.042124) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.895822 (0.049798) with: {'learn_rate': 0

# Experimenting with autoML libraries 

## TPOT pipeline creation

In [ ]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier()
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))

## PyCaret classifier creation

In [ ]:
from pycaret.datasets import get_data
data = new_train_df
from pycaret.classification import *
exp_name = setup(data = new_train_df,target = 'outcome')

In [ ]:
best_model = compare_models()


In [ ]:
best_model.get_params()

In [ ]:
from pycaret.datasets import get_data


from pycaret.classification import *

exp_name = setup(data = data,  target = 'outcome',silent=True)

cb = create_model('catboost')

optimize_threshold(cb, true_negative = 10, false_negative = -100)


In [ ]:
cb.get_params()